In [1]:
import dask.dataframe as dd
import pandas as pd

import dask_ml.cluster

In [2]:
import matplotlib.pyplot as plt

In [3]:
import itertools

In [4]:
import tqdm

In [5]:
def get_clusters_on(ddf, columns, upper_n_clusters):

    data = ddf[columns].compute()
    for n_clusters in range(2, upper_n_clusters):
        kmeans = dask_ml.cluster.KMeans(n_clusters=n_clusters, random_state=0).fit(data)
        ddf['Clusters on {} - {}: n = {}'.format(*columns_to_cluster, n_clusters)] = dd.from_dask_array(kmeans.labels_)
    
    return ddf

**Read data**

In [5]:
df = pd.read_csv('../data/processed_data_sample.csv')

In [6]:
ddf = dd.from_pandas(df, npartitions=16)

In [7]:
# ddf = ddf.fillna(0)

**Clustering Tuples**

In [10]:
columns = ddf.columns[1:].drop(['Memory_depth',
                      'Unnamed: 0.1',
                      'Unnamed: 0.1.1',
                      'Unnamed: 0.1.1.1',
                      'Name', 'Rank', 'seed',
                      'Makes_use_of_game',
                      'Makes_use_of_length',
                      'Unnamed: 0.1.1.1.1',
                               'Stochastic'])

In [11]:
columns

Index(['CC_rate', 'CC_to_C_rate', 'CD_rate', 'CD_to_C_rate',
       'Cooperation_rating', 'DC_rate', 'DC_to_C_rate', 'DD_rate',
       'DD_to_C_rate', 'Initial_C_rate', 'Median_score', 'Normalized_Rank',
       'SSeeror', 'Wins', 'noise', 'probend', 'repetitions', 'size', 'turns'],
      dtype='object')

In [12]:
columns_to_cluster = itertools.combinations(columns, 2)

In [13]:
for column_one, column_two in columns_to_cluster:
    #ddf = get_clusters_on(ddf, list(columns), 4)
    
    sample = ddf.sample(frac=0.01)
    
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(5, 4))

    x = sample[column_one].compute()
    y = sample[column_two].compute()

    
    axes.scatter(x, y,);

    fig.suptitle('Scatter on {} - {}'.format(column_one, column_two));
    fig.savefig('../images/clustering/{}_{}.png'.format(column_one, column_two));
    plt.close();